In [1]:
%cd ../
import numpy as np
import pandas as pd
from tqdm import tqdm
%load_ext autoreload
%autoreload 2

/home/harshapk/Projects/CSS_Project


In [23]:
import pickle
with open("./data/data_processed/covid_lies_dataset.pkl", "rb") as f:
    dataset = pickle.load(f)

In [24]:
dataset = dataset[dataset["lang"] == "en"]

In [25]:
dataset = dataset[["tweet_text", "retweets", "likes", "label"]]

In [26]:
dataset["label"].unique()

array(['na', 'neg', 'pos'], dtype=object)

In [ ]:
from css_proj.models.bertembed import get_sentence_embed
import transformers as ppb
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
all_tw_text = dataset["tweet_text"].tolist()
all_tw_text_embed = [get_sentence_embed([text], model, tokenizer).numpy() for text in tqdm(all_tw_text)]

In [ ]:
all_tw_text_embed = np.array(all_tw_text_embed)
all_tw_text_embed = all_tw_text_embed.squeeze()

In [ ]:
all_tw_desc = dataset["user_description"].tolist()
all_tw_desc_embed = [get_sentence_embed([text], model, tokenizer).numpy() for text in tqdm(all_tw_desc)]
all_tw_desc_embed = np.array(all_tw_desc_embed).squeeze()

In [ ]:
with open("./data/data_processed/covid_lies_embed.pkl", "wb") as f:
    pickle.dump([all_tw_text_embed, all_tw_desc_embed], f)

In [27]:
with open("./data/data_processed/covid_lies_embed.pkl", "rb") as f:
    all_tw_text_embed, all_tw_desc_embed = pickle.load(f)

In [28]:
dataset["num_label"] = dataset["label"].apply(lambda x: 0 if x == "na" else 1)

In [29]:
dataset.head()

,tweet_text,retweets,likes,label,num_label
0,Getting coronavirus and then coughing on peopl...,1,0,na,0
4,"“Most people (not elderly, no underlying healt...",0,0,na,0
5,15-20% of people with COVID-19 may require hos...,0,0,na,0
6,A sheriff's office tried to get meth off the s...,0,0,na,0
7,"In China’s War on the Coronavirus, a Community...",0,0,na,0


In [30]:
dataset_array = dataset[["retweets", "likes", "num_label"]]
dataset_array = np.array(dataset_array).astype(float)

In [31]:
dataset_array = np.hstack([all_tw_desc_embed, all_tw_text_embed, dataset_array])
dataset_x = dataset_array[:, :-1]
dataset_y = dataset_array[:, -1]

In [32]:
# Train test split
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(dataset_x, dataset_y, test_size=0.4, random_state=42)

In [33]:
# Normalize features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

In [34]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(train_x, train_y)

/home/harshapk/Projects/CSS_Project/.envs/css_proj/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [35]:
# Evaluate model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
pred_y = logreg.predict(test_x)
print(accuracy_score(test_y, pred_y))
print(confusion_matrix(test_y, pred_y))
print(classification_report(test_y, pred_y))
print(f1_score(test_y, pred_y))

0.8924050632911392
[[281  14]
 [ 20   1]]
              precision    recall  f1-score   support

         0.0       0.93      0.95      0.94       295
         1.0       0.07      0.05      0.06        21

    accuracy                           0.89       316
   macro avg       0.50      0.50      0.50       316
weighted avg       0.88      0.89      0.88       316

0.05555555555555555


In [36]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(train_x, train_y)

RandomForestClassifier(random_state=42)

In [37]:
# Evaluate model
pred_y = rfc.predict(test_x)
print(accuracy_score(test_y, pred_y))
print(confusion_matrix(test_y, pred_y))
print(classification_report(test_y, pred_y))
print(f1_score(test_y, pred_y))

0.9240506329113924
[[292   3]
 [ 21   0]]
              precision    recall  f1-score   support

         0.0       0.93      0.99      0.96       295
         1.0       0.00      0.00      0.00        21

    accuracy                           0.92       316
   macro avg       0.47      0.49      0.48       316
weighted avg       0.87      0.92      0.90       316

0.0


In [38]:
# SVM
from sklearn.svm import SVC
svc = SVC(kernel="linear", random_state=42)
svc.fit(train_x, train_y)

SVC(kernel='linear', random_state=42)

In [39]:
# Evaluate model
pred_y = svc.predict(test_x)
print(accuracy_score(test_y, pred_y))
print(confusion_matrix(test_y, pred_y))
print(classification_report(test_y, pred_y))
print(f1_score(test_y, pred_y))

0.870253164556962
[[272  23]
 [ 18   3]]
              precision    recall  f1-score   support

         0.0       0.94      0.92      0.93       295
         1.0       0.12      0.14      0.13        21

    accuracy                           0.87       316
   macro avg       0.53      0.53      0.53       316
weighted avg       0.88      0.87      0.88       316

0.12765957446808512


In [40]:
# SVM with RBF kernel
svc_rbf = SVC(kernel="rbf", random_state=42)
svc_rbf.fit(train_x, train_y)

SVC(random_state=42)

In [41]:
# Evaluate model
pred_y = svc_rbf.predict(test_x)
print(accuracy_score(test_y, pred_y))
print(confusion_matrix(test_y, pred_y))
print(classification_report(test_y, pred_y))
print(f1_score(test_y, pred_y))

0.9335443037974683
[[295   0]
 [ 21   0]]
              precision    recall  f1-score   support

         0.0       0.93      1.00      0.97       295
         1.0       0.00      0.00      0.00        21

    accuracy                           0.93       316
   macro avg       0.47      0.50      0.48       316
weighted avg       0.87      0.93      0.90       316

0.0


/home/harshapk/Projects/CSS_Project/.envs/css_proj/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/harshapk/Projects/CSS_Project/.envs/css_proj/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/harshapk/Projects/CSS_Project/.envs/css_proj/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [42]:
# Neural Network
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(random_state=42, hidden_layer_sizes=(100, 100, 100))
mlp.fit(train_x, train_y)

MLPClassifier(hidden_layer_sizes=(100, 100, 100), random_state=42)

In [43]:
# Evaluate model
pred_y = mlp.predict(test_x)
print(accuracy_score(test_y, pred_y))
print(confusion_matrix(test_y, pred_y))
print(classification_report(test_y, pred_y))
print(f1_score(test_y, pred_y))

0.9145569620253164
[[288   7]
 [ 20   1]]
              precision    recall  f1-score   support

         0.0       0.94      0.98      0.96       295
         1.0       0.12      0.05      0.07        21

    accuracy                           0.91       316
   macro avg       0.53      0.51      0.51       316
weighted avg       0.88      0.91      0.90       316

0.06896551724137931
